In [ ]:
try:
            import traci
            print("  ✓ TraCI (SUMO Python API)")
except ImportError:
            print("  ✗ TraCI not available")
            self.warnings.append("TraCI not available - SUMO integration may not work")

return all_good

    def fix_common_errors(self):
        """Provide fixes for common installation errors"""
        print("\n" + "="*50)
        print("Common Error Fixes")
        print("="*50)

        fixes = {
            "gymnasium": [
                "If gymnasium fails, try:",
                "  pip install gym",
                "  pip install gymnasium --upgrade"
            ],
            "stable-baselines3": [
                "If stable-baselines3 fails, try:",
                "  pip install torch",
                "  pip install stable-baselines3 --no-deps",
                "  pip install stable-baselines3"
            ],
            "sumo-rl": [
                "If sumo-rl fails, try:",
                "  pip install traci",
                "  pip install sumo-rl --no-deps",
                "Or clone from GitHub:",
                "  git clone https://github.com/LucasAlegre/sumo-rl",
                "  cd sumo-rl && pip install -e ."
            ],
            "SUMO": [
                "If SUMO is not found:",
                "1. Ensure SUMO is installed",
                "2. Find SUMO installation directory",
                "3. Set SUMO_HOME environment variable",
                "4. Add SUMO/bin to PATH"
            ]
        }

        for error_type, fix_steps in fixes.items():
            print(f"\n{error_type}:")
            for step in fix_steps:
                print(f"  {step}")

    def create_test_script(self):
        """Create a test script to verify the installation"""
        test_script = '''#!/usr/bin/env python3
"""
Test script for Smart Traffic Management System
Run this to verify your installation is working
"""

import sys
import os

def test_imports():
    """Test if all required packages can be imported"""
    print("Testing package imports...")

    packages = [
        'numpy',
        'gymnasium',
        'stable_baselines3',
        'matplotlib',
        'pandas',
        'tensorboard'
    ]

    failed = []
    for package in packages:
        try:
            __import__(package)
            print(f"  ✓ {package}")
        except ImportError as e:
            print(f"  ✗ {package}: {e}")
            failed.append(package)

    # Test SUMO-RL
    try:
        import sumo_rl
        print(f"  ✓ sumo-rl")
    except ImportError:
        print(f"  ⚠ sumo-rl (optional)")

    # Test TraCI
    try:
        import traci
        print(f"  ✓ traci")
    except ImportError:
        print(f"  ⚠ traci (needed for SUMO)")

    return len(failed) == 0

def test_sumo():
    """Test SUMO configuration"""
    print("\\nTesting SUMO configuration...")

    if 'SUMO_HOME' in os.environ:
        print(f"  ✓ SUMO_HOME = {os.environ['SUMO_HOME']}")
        return True
    else:
        print("  ✗ SUMO_HOME not set")
        return False

def test_rl_environment():
    """Test creating a simple RL environment"""
    print("\\nTesting RL environment creation...")

    try:
        import gymnasium as gym
        from stable_baselines3 import PPO

        # Create a simple environment
        env = gym.make('CartPole-v1')
        print("  ✓ Gymnasium environment created")

        # Create a PPO model
        model = PPO('MlpPolicy', env, verbose=0)
        print("  ✓ PPO model created")

        env.close()
        return True
    except Exception as e:
        print(f"  ✗ Failed: {e}")
        return False

if __name__ == "__main__":
    print("="*50)
    print("Smart Traffic Management System - Installation Test")
    print("="*50)

    results = []
    results.append(test_imports())
    results.append(test_sumo())
    results.append(test_rl_environment())

    print("\\n" + "="*50)
    if all(results):
        print("✅ All tests passed! Your system is ready.")
    else:
        print("❌ Some tests failed. Please check the errors above.")
    print("="*50)
'''

        with open('test_installation.py', 'w') as f:
            f.write(test_script)

        print("\nCreated test_installation.py")
        print("Run it with: python test_installation.py")

    def run(self):
        """Run the complete installation process"""
        print("="*60)
        print(" Smart Traffic Management System - Installation Helper")
        print(" Problem ID: 25050 - Government of Odisha")
        print("="*60)

IndentationError: unindent does not match any outer indentation level (<string>, line 10)

In [8]:
"""
SUMO Traffic Light Control Environment with RL
Phase 3 Implementation for Urban Traffic Management
"""

import os
import sys
import numpy as np
import gymnasium as gym
from gymnasium import spaces
import traci
import sumo_rl
from stable_baselines3 import PPO, DQN
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback
import matplotlib.pyplot as plt
from datetime import datetime

class TrafficSignalEnv(gym.Env):
    """
    Custom SUMO Environment for Traffic Signal Control
    Optimizes signal timings to reduce congestion
    """

    def __init__(self, net_file, route_file, use_gui=False, num_seconds=3600):
        super(TrafficSignalEnv, self).__init__()

        self.net_file = net_file
        self.route_file = route_file
        self.use_gui = use_gui
        self.num_seconds = num_seconds

        # Traffic light phases
        self.phases = [
            'GGrrrrGGrrrr',  # NS straight and right
            'yyrrrryyrrr',   # NS yellow
            'rrGGGGrrGGGG',  # EW straight and right
            'rryyyyyyyy'     # EW yellow
        ]

        # Action space: 4 possible phases
        self.action_space = spaces.Discrete(4)

        # Observation space:
        # - Queue lengths (4 directions)
        # - Waiting times (4 directions)
        # - Current phase
        # - Time since phase change
        self.observation_space = spaces.Box(
            low=0, high=100, shape=(10,), dtype=np.float32
        )

        self.sumo_cmd = None
        self.connection = None

    def reset(self, seed=None, options=None):
        """Reset the environment"""
        super().reset(seed=seed)

        # Close existing SUMO instance
        if self.connection:
            traci.close()

        # Start SUMO
        if self.use_gui:
            sumo_binary = "sumo-gui"
        else:
            sumo_binary = "sumo"

        self.sumo_cmd = [
            sumo_binary,
            "-n", self.net_file,
            "-r", self.route_file,
            "--no-warnings",
            "--no-step-log",
            "--random"
        ]

        traci.start(self.sumo_cmd)
        self.connection = traci

        # Initialize metrics
        self.current_phase = 0
        self.phase_duration = 0
        self.total_waiting_time = 0
        self.total_queue_length = 0

        return self._get_observation(), {}

    def step(self, action):
        """Execute one step in the environment"""
        # Change traffic light phase if needed
        if action != self.current_phase:
            self._set_traffic_phase(action)
            self.current_phase = action
            self.phase_duration = 0
        else:
            self.phase_duration += 1

        # Simulate one step
        self.connection.simulationStep()

        # Get observation
        observation = self._get_observation()

        # Calculate reward
        reward = self._calculate_reward()

        # Check if simulation is done
        done = self.connection.simulation.getTime() >= self.num_seconds

        return observation, reward, done, False, {}

    def _get_observation(self):
        """Get current state observation"""
        obs = []

        # Get queue lengths for each lane
        lanes = self.connection.lane.getIDList()[:4]  # Get first 4 lanes
        for lane in lanes:
            queue_length = self.connection.lane.getLastStepHaltingNumber(lane)
            obs.append(min(queue_length, 50))  # Cap at 50

        # Get waiting times
        for lane in lanes:
            waiting_time = self.connection.lane.getWaitingTime(lane)
            obs.append(min(waiting_time, 100))  # Cap at 100

        # Current phase and duration
        obs.append(self.current_phase)
        obs.append(min(self.phase_duration, 60))  # Cap at 60

        return np.array(obs, dtype=np.float32)

    def _calculate_reward(self):
        """
        Calculate reward based on:
        - Queue length reduction
        - Waiting time reduction
        - Throughput improvement
        """
        current_queue = 0
        current_waiting = 0

        for lane in self.connection.lane.getIDList()[:4]:
            current_queue += self.connection.lane.getLastStepHaltingNumber(lane)
            current_waiting += self.connection.lane.getWaitingTime(lane)

        # Negative reward for high queue and waiting time
        queue_penalty = -current_queue * 0.5
        waiting_penalty = -current_waiting * 0.01

        # Positive reward for vehicles passing
        throughput = self.connection.simulation.getArrivedNumber()
        throughput_reward = throughput * 1.0

        reward = queue_penalty + waiting_penalty + throughput_reward

        # Store for next step comparison
        self.total_queue_length = current_queue
        self.total_waiting_time = current_waiting

        return reward

    def _set_traffic_phase(self, phase_index):
        """Set traffic light to specific phase"""
        traffic_lights = self.connection.trafficlight.getIDList()
        if traffic_lights:
            tl_id = traffic_lights[0]
            self.connection.trafficlight.setRedYellowGreenState(
                tl_id, self.phases[phase_index]
            )

    def close(self):
        """Close the environment"""
        if self.connection:
            traci.close()

    def render(self, mode='human'):
        """Render is handled by SUMO GUI if enabled"""
        pass


class TrafficRLTrainer:
    """
    Main trainer class for RL-based traffic control
    Supports PPO and DQN algorithms
    """

    def __init__(self, algorithm='PPO', config=None):
        self.algorithm = algorithm
        self.config = config or self._get_default_config()
        self.model = None
        self.env = None

    def _get_default_config(self):
        """Default configuration for RL training"""
        return {
            'learning_rate': 3e-4,
            'n_steps': 2048,
            'batch_size': 64,
            'n_epochs': 10,
            'gamma': 0.99,
            'gae_lambda': 0.95,
            'clip_range': 0.2,
            'ent_coef': 0.01,
            'tensorboard_log': './traffic_rl_logs/',
            'verbose': 1
        }

    def create_environment(self, net_file, route_file, use_gui=False):
        """Create SUMO environment"""
        def make_env():
            return TrafficSignalEnv(net_file, route_file, use_gui)

        # Use vectorized environment for parallel training
        self.env = DummyVecEnv([make_env])
        return self.env

    def train(self, total_timesteps=100000, save_path='traffic_rl_model'):
        """Train the RL model"""
        if self.env is None:
            raise ValueError("Environment not created. Call create_environment first.")

        # Select algorithm
        if self.algorithm == 'PPO':
            self.model = PPO(
                'MlpPolicy',
                self.env,
                learning_rate=self.config['learning_rate'],
                n_steps=self.config['n_steps'],
                batch_size=self.config['batch_size'],
                n_epochs=self.config['n_epochs'],
                gamma=self.config['gamma'],
                gae_lambda=self.config['gae_lambda'],
                clip_range=self.config['clip_range'],
                ent_coef=self.config['ent_coef'],
                tensorboard_log=self.config['tensorboard_log'],
                verbose=self.config['verbose']
            )
        elif self.algorithm == 'DQN':
            self.model = DQN(
                'MlpPolicy',
                self.env,
                learning_rate=self.config['learning_rate'],
                buffer_size=50000,
                learning_starts=1000,
                batch_size=32,
                tau=1.0,
                gamma=self.config['gamma'],
                train_freq=4,
                gradient_steps=1,
                target_update_interval=1000,
                tensorboard_log=self.config['tensorboard_log'],
                verbose=self.config['verbose']
            )
        else:
            raise ValueError(f"Unsupported algorithm: {self.algorithm}")

        # Setup callbacks
        eval_callback = EvalCallback(
            self.env,
            best_model_save_path=f'./{save_path}_best/',
            log_path=f'./{save_path}_logs/',
            eval_freq=5000,
            deterministic=True,
            render=False
        )

        checkpoint_callback = CheckpointCallback(
            save_freq=10000,
            save_path=f'./{save_path}_checkpoints/',
            name_prefix='rl_model'
        )

        # Train the model
        print(f"Starting training with {self.algorithm}...")
        self.model.learn(
            total_timesteps=total_timesteps,
            callback=[eval_callback, checkpoint_callback]
        )

        # Save final model
        self.model.save(f"{save_path}_final")
        print(f"Training completed. Model saved to {save_path}_final")

        return self.model

    def evaluate(self, n_eval_episodes=10):
        """Evaluate the trained model"""
        if self.model is None:
            raise ValueError("No model trained. Train a model first.")

        mean_reward, std_reward = evaluate_policy(
            self.model,
            self.env,
            n_eval_episodes=n_eval_episodes,
            render=False
        )

        print(f"Evaluation results over {n_eval_episodes} episodes:")
        print(f"Mean reward: {mean_reward:.2f} +/- {std_reward:.2f}")

        return mean_reward, std_reward

    def predict(self, observation):
        """Predict action for given observation"""
        if self.model is None:
            raise ValueError("No model trained. Train or load a model first.")

        action, _states = self.model.predict(observation, deterministic=True)
        return action

    def load_model(self, model_path):
        """Load a pre-trained model"""
        if self.algorithm == 'PPO':
            self.model = PPO.load(model_path)
        elif self.algorithm == 'DQN':
            self.model = DQN.load(model_path)
        print(f"Model loaded from {model_path}")

    def plot_training_curves(self, log_path='./traffic_rl_logs/'):
        """Plot training curves from tensorboard logs"""
        # This would require parsing tensorboard logs
        # For now, providing a placeholder
        print("Use 'tensorboard --logdir=./traffic_rl_logs/' to view training curves")


# Utility function to create sample SUMO files
def create_sample_sumo_files(output_dir='./sumo_files/'):
    """Create sample SUMO network and route files for testing"""
    os.makedirs(output_dir, exist_ok=True)

    # Sample network file (4-way intersection)
    net_content = """<?xml version="1.0" encoding="UTF-8"?>
<net version="1.16" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
    <junction id="J0" type="traffic_light" x="0.00" y="0.00">
        <request index="0" response="0000000000000000" foes="0000000000000000"/>
    </junction>
</net>"""

    # Sample route file
    route_content = """<?xml version="1.0" encoding="UTF-8"?>
<routes>
    <vType id="car" accel="2.6" decel="4.5" length="5" maxSpeed="55.55"/>
    <flow id="flow_NS" type="car" begin="0" end="3600" probability="0.3" from="north" to="south"/>
    <flow id="flow_SN" type="car" begin="0" end="3600" probability="0.3" from="south" to="north"/>
    <flow id="flow_EW" type="car" begin="0" end="3600" probability="0.2" from="east" to="west"/>
    <flow id="flow_WE" type="car" begin="0" end="3600" probability="0.2" from="west" to="east"/>
</routes>"""

    with open(f"{output_dir}/network.net.xml", 'w') as f:
        f.write(net_content)

    with open(f"{output_dir}/routes.rou.xml", 'w') as f:
        f.write(route_content)

    print(f"Sample SUMO files created in {output_dir}")
    return f"{output_dir}/network.net.xml", f"{output_dir}/routes.rou.xml"